In [4]:
import os
import json
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

os.environ["PYSPARK_PYTHON"] = "/srv/spark/.venv/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/srv/spark/.venv/bin/python"
HIVE_URI = os.environ.get("ARCTIC_URI") ## Hive Server URI


def get_spark_session(app_name: str, conf_path: str = "spark.json") -> SparkSession:
    spark = SparkSession.builder.appName(app_name)
    with open(conf_path, "r") as conf_file:
        cfg = json.loads(conf_file.read())
        spark = spark.master(cfg["master"]) \
                .config("spark.authenticate", "true") \
                .config("spark.authenticate.secret", cfg["secret"]) \
                .config("spark.executor.memory", "8g")\
                .config('spark.sql.catalog.iceberg.warehouse', '/srv/storage/test_iceberg')\
                .config('spark.sql.catalog.iceberg.type', 'hadoop')\
                .config("spark.sql.catalog.default", "iceberg") \
                .config('spark.sql.catalog.iceberg', 'org.apache.iceberg.spark.SparkCatalog')\
                .config('spark.sql.extensions','org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')\
                .config('spark.jars.packages','org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.0.0')\
                .config('spark.sql.warehouse.dir','/srv/storage/test_iceberg_hive')\
                .config('spark.sql.catalog.hive', 'org.apache.iceberg.spark.SparkCatalog')\
                .config('spark.sql.catalog.hive.type', 'hadoop')\
                .config('spark.sql.catalog.hive.uri', HIVE_URI)
    return spark.getOrCreate()
spark = get_spark_session("jupyter-fichier-Iceberg")
print("Spark Running")

:: loading settings :: url = jar:file:/srv/jupyterhub/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/nseb/.ivy2/cache
The jars for the packages stored in: /home/nseb/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.3_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-35f08aa1-ad61-4cfe-b53c-6220802619dd;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.0.0 in central
:: resolution report :: resolve 96ms :: artifacts dl 3ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.0.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#s

23/07/05 10:07:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark Running


In [2]:
parquet_files = "/srv/parquetevents/mongo_parquet/reunion/2022-09-*-reunion.snappy.parquet"

df = spark.read.parquet(parquet_files).select(
    f.col("_id").alias("id"), "date", f.col("userId").alias("user_id"),
    f.col("profil").alias("profile"),f.col("event-type").alias("event_type"),
    "module","ua", f.col("resource-type").alias("resource_type"))


df.write.format("iceberg").save("/srv/storage/iceberg/events")
df.show()

23/07/04 15:55:32 WARN Query: Query for candidates of org.apache.hadoop.hive.metastore.model.MDatabase and subclasses resulted in no possible candidates
Required table missing : "DBS" in Catalog "" Schema "". DataNucleus requires this table to perform its persistence operations. Either your MetaData is incorrect, or you need to enable "datanucleus.schema.autoCreateTables"
org.datanucleus.store.rdbms.exceptions.MissingTableException: Required table missing : "DBS" in Catalog "" Schema "". DataNucleus requires this table to perform its persistence operations. Either your MetaData is incorrect, or you need to enable "datanucleus.schema.autoCreateTables"
	at org.datanucleus.store.rdbms.table.AbstractTable.exists(AbstractTable.java:606)
	at org.datanucleus.store.rdbms.RDBMSStoreManager$ClassAdder.performTablesValidation(RDBMSStoreManager.java:3385)
	at org.datanucleus.store.rdbms.RDBMSStoreManager$ClassAdder.run(RDBMSStoreManager.java:2896)
	at org.datanucleus.store.rdbms.AbstractSchemaTran

Py4JJavaError: An error occurred while calling o76.save.
: org.apache.iceberg.hive.RuntimeMetaException: Failed to connect to Hive Metastore
	at org.apache.iceberg.hive.HiveClientPool.newClient(HiveClientPool.java:84)
	at org.apache.iceberg.hive.HiveClientPool.newClient(HiveClientPool.java:34)
	at org.apache.iceberg.ClientPoolImpl.get(ClientPoolImpl.java:125)
	at org.apache.iceberg.ClientPoolImpl.run(ClientPoolImpl.java:56)
	at org.apache.iceberg.ClientPoolImpl.run(ClientPoolImpl.java:51)
	at org.apache.iceberg.hive.CachedClientPool.run(CachedClientPool.java:82)
	at org.apache.iceberg.hive.HiveTableOperations.doRefresh(HiveTableOperations.java:205)
	at org.apache.iceberg.BaseMetastoreTableOperations.refresh(BaseMetastoreTableOperations.java:95)
	at org.apache.iceberg.BaseMetastoreTableOperations.current(BaseMetastoreTableOperations.java:78)
	at org.apache.iceberg.BaseMetastoreCatalog.loadTable(BaseMetastoreCatalog.java:43)
	at org.apache.iceberg.spark.SparkCatalog.load(SparkCatalog.jav

In [8]:
## Requête create table
#spark.sql("DROP TABLE iceberg.table1")
spark.sql("DROP TABLE iceberg.statistiques")
spark.sql("CREATE TABLE iceberg.statistiques(connection_type string,device_type string)")

## Requête insert dans la table 1
spark.sql("INSERT INTO iceberg.statistiques (connection_type, device_type) VALUES ('LOGIN','DESKTOP')")
spark.sql("INSERT INTO iceberg.statistiques (connection_type, device_type) VALUES ('LOGIN','MOBILE')")
spark.sql("INSERT INTO iceberg.statistiques (connection_type, device_type) VALUES ('ACTIVATION','DESKTOP')")
spark.sql("INSERT INTO iceberg.statistiques (connection_type, device_type) VALUES ('ACTIVATION','LAPTOP')")
spark.sql("INSERT INTO iceberg.statistiques (connection_type, device_type) VALUES ('LOGIN','DESKTOP')")


## Requête select
df = spark.sql("SELECT * FROM iceberg.statistiques WHERE iceberg.statistiques.connection_type='LOGIN'")

## Affichage de la table
df.show()

##arrêt de spark
spark.stop()


23/07/05 10:08:53 WARN FileUtil: Failed to delete file or dir [/srv/storage/test_iceberg/statistiques/data/00000-3-8c27d4a2-1ca8-4365-8093-206ac74d4847-00001.parquet]: it still exists.
23/07/05 10:08:53 WARN FileUtil: Failed to delete file or dir [/srv/storage/test_iceberg/statistiques/data/.00000-3-add4bd02-3005-4872-b3fb-c59a379c017e-00001.parquet.crc]: it still exists.
23/07/05 10:08:53 WARN FileUtil: Failed to delete file or dir [/srv/storage/test_iceberg/statistiques/data/.00000-2-86ce5c7d-7528-4997-ab72-bc2bd9002f5d-00001.parquet.crc]: it still exists.
23/07/05 10:08:53 WARN FileUtil: Failed to delete file or dir [/srv/storage/test_iceberg/statistiques/data/.00000-3-a27cb9a9-4349-451f-8c86-ec44143dbbdd-00001.parquet.crc]: it still exists.
23/07/05 10:08:53 WARN FileUtil: Failed to delete file or dir [/srv/storage/test_iceberg/statistiques/data/.00000-4-4bc7f870-a878-48b2-9630-8a93a584d2ad-00001.parquet.crc]: it still exists.
23/07/05 10:08:53 WARN FileUtil: Failed to delete file 

In [17]:
##importer la fonction Hive (partition evolution)
import pyspark
from pyspark.sql import SparkSession
import os

## DEFINE SENSITIVE VARIABLES
HIVE_URI = os.environ.get("ARCTIC_URI") ## Hive Server URI

conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')
  		#Configuring Catalog
        .set('spark.sql.catalog.hive', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.hive.type', 'hadoop')
        .set('spark.sql.catalog.hive.warehouse', '/srv/storage/fichier_log/travail')
        .set('spark.sql.catalog.hive.uri', HIVE_URI)
        .set('spark.sql.catalog.hive.io-impl', '/srv/storage/fichier_log/travail')
  		)

In [40]:
## Démararage sparksession
spark = get_spark_session("jupyter-fichier-Iceberg")
print("Spark Running")

## DataFrame depuis parquet
dataframe=spark.read.parquet("/srv/parquetevents/mongo_parquet/paris/2022-11-30-paris.snappy.parquet")

dataframe.createOrReplaceTempView("myview")
dataframe.show()

23/07/05 14:32:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
Spark Running
+--------------------+----------+--------------------+-------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------+--------+-----------------+-------+---------+--------+-----------+------+---------+-------+-------+------------+---------+---------+--------+-------+--------+------+------+
|                 _id|event-type|              module|         date|              userId|  profil|          structures|             classes|              groups|                  ua|            ip|             service|connector-type|cas-type|    resource-type|referer|sessionId|video_id|device_type|source|useradmin|adapter|nbUsers|nbStructures|nbClasses|old-users|new-user|session|duration|weight|result|
+--------------------+----------+--------------------+

In [42]:
spark.sql("CREATE SCHEMA my_iceberg")

DataFrame[]

In [ ]:
spark.sql("SHOW ERRORS")

In [50]:
## verifie si table1 existe et la supprime dans ce cas la 
spark.sql("DROP TABLE IF EXISTS iceberg.table1")

##creer table
spark.sql("CREATE TABLE iceberg.table1 AS (SELECT * FROM myview)")

## upsert avec merge
spark.sql("""
MERGE INTO iceberg.table1 AS t
    USING (SELECT * FROM myview) AS s
    ON t._id = s._id
    WHEN MATCHED THEN DELETE
    WHEN NOT MATCHED THEN INSERT *
""")

23/07/05 16:36:35 ERROR AtomicCreateTableAsSelectExec: Data source write support IcebergBatchWrite(table=table1, format=PARQUET) is aborting.
23/07/05 16:36:35 ERROR AtomicCreateTableAsSelectExec: Data source write support IcebergBatchWrite(table=table1, format=PARQUET) aborted.
23/07/05 16:36:35 ERROR Utils: Aborting task
org.apache.spark.SparkException: Writing job aborted
	at org.apache.spark.sql.errors.QueryExecutionErrors$.writingJobAbortedError(QueryExecutionErrors.scala:767)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2(WriteToDataSourceV2Exec.scala:409)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2$(WriteToDataSourceV2Exec.scala:353)
	at org.apache.spark.sql.execution.datasources.v2.AtomicCreateTableAsSelectExec.writeWithV2(WriteToDataSourceV2Exec.scala:108)
	at org.apache.spark.sql.execution.datasources.v2.TableWriteExecHelper.$anonfun$writeToTable$1(WriteToDataSourceV2Exec.scala:503)
	at org.apache.spark.util.U

Py4JJavaError: An error occurred while calling o125.sql.
: org.apache.spark.SparkException: Writing job aborted
	at org.apache.spark.sql.errors.QueryExecutionErrors$.writingJobAbortedError(QueryExecutionErrors.scala:767)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2(WriteToDataSourceV2Exec.scala:409)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2$(WriteToDataSourceV2Exec.scala:353)
	at org.apache.spark.sql.execution.datasources.v2.AtomicCreateTableAsSelectExec.writeWithV2(WriteToDataSourceV2Exec.scala:108)
	at org.apache.spark.sql.execution.datasources.v2.TableWriteExecHelper.$anonfun$writeToTable$1(WriteToDataSourceV2Exec.scala:503)
	at org.apache.spark.util.Utils$.tryWithSafeFinallyAndFailureCallbacks(Utils.scala:1538)
	at org.apache.spark.sql.execution.datasources.v2.TableWriteExecHelper.writeToTable(WriteToDataSourceV2Exec.scala:491)
	at org.apache.spark.sql.execution.datasources.v2.TableWriteExecHelper.writeToTable$(

In [47]:
spark.sql("SELECT * FROM iceberg.table1").show()
spark.sql("SELECT COUNT(*) FROM iceberg.table1 WHERE iceberg.table1.module='SchoolBook'").show()

AnalysisException: Unsupported data source type for direct query on files: iceberg; line 1 pos 14